In [28]:
from fastai.conv_learner import *
from torch.nn import *
import torchvision
import torchvision.transforms as transforms

In [29]:
PATH = 'data/'
sz = 224
bs = 64

In [32]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
train_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [33]:
class SimpleNet(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(layers[i], layers[i + 1]) for i in range(len(layers) - 1)])
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        for l in self.layers:
            l_x = l(x)
            x = F.relu(l_x)
        return F.log_softmax(l_x, dim=-1)

In [34]:
model = SimpleNet([sz*sz*3,40,10]).cuda()
opt = optim.Adam(model.parameters(), 1e-2)

In [36]:
fit(model, train_data , 8 , opt, F.cross_entropy)

epoch      trn_loss   val_loss                          
    0      53.703886  67.834496 
    1      46.494749  33.659956                         
    2      37.927877  17.472156                         
    3      31.078512  21.976673                         
    4      25.693452  11.443785                         
    5      21.578327  14.427269                         
    6      18.372324  8.945817                          
    7      15.77884   8.860509                          



[array([8.86051])]

In [40]:
###TEST A HOT DOG IMAGE
imsize = 256
loader = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

In [45]:
test_image_path = 'data/test/hot_dog/133012.jpg'
image = image_loader(test_image_path)

y = SimpleNet(image)

SimpleNet(
  (layers): ModuleList(
  )
)